In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Nov 18 11:14:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   27C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 7.9 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 3.5 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 2.4 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-bw10vn5y
  Running command git clone -q https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-bw10vn5y
  Created wheel for torch-geomet

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 72.8 MB/s 
     |████████████████████████████████| 182 kB 87.4 MB/s 


In [6]:
import pickle
import torch
import transformers
from transformers import BertTokenizer, BertModel

from transformers import AutoTokenizer, AutoModel, AutoConfig


from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import InMemoryDataset, download_url
from torch_geometric.data import Data

from torch_geometric.loader import DataLoader

import torch.nn.functional as F
import numpy as np

import math
import pandas as pd
import pip
import re

import os.path as osp

import torch
from torch_geometric.data import Dataset, download_url

import random

random.seed(123)

torch.manual_seed(123)
np.random.seed(123)
torch.cuda.manual_seed_all(123)
from sklearn.metrics import jaccard_score, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix


In [7]:
with open("/content/drive/MyDrive/Colab Notebooks/27/full_dialog/one_disease/final_combined_all_dialog.p", "rb") as f:
    final_dialog_sheet = pickle.load(f)

In [8]:
final_dialog_sheet

[{'dialog': "[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK].",
  'label': 'central retinal artery or vein occlusion'},
 {'dialog': "[PAT] : Doctor, I have Shoulder pain. I feel terrible. Would you please tell me what could be its cause? [SEP] [DOC] :  I can feel the pain you feel. Let me ask you some questions. Do you have Back pain? [SEP] [PAT] : Literally, I am suffering from Back pain. [SEP] [DOC] : Do you have Low back pain? [SEP] [PAT] : Yes, I am dealing with Low back pain. [SEP] [DOC] : Are you also d

In [9]:

total_dialog_sheet = final_dialog_sheet
len(total_dialog_sheet)

1367

In [10]:
with open("/content/drive/MyDrive/Colab Notebooks/27/self_report/one_disease/final_combined_all_self_report.p", "rb") as f:
  self_report = pickle.load(f)

In [11]:
self_report

[{'first_utterance': "[SR_START] [PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SR_END]",
  'label': 'central retinal artery or vein occlusion'},
 {'first_utterance': '[SR_START] [PAT] : Doctor, I have Shoulder pain. I feel terrible. Would you please tell me what could be its cause? [SR_END]',
  'label': 'degenerative disc disease'},
 {'first_utterance': '[SR_START] [PAT] : Hi! Doctor, I am suffering from Foreign body sensation in eye. Would you please tell me what could be its cause? [SR_END]',
  'label': 'diabetic retinopathy'},
 {'first_utterance': "[SR_START] [PAT] : Doctor, I am dealing with Low back pain. I can't sleep well because of this, and in the daytime feels lethargic. Please help me.  [SR_END]",
  'label': 'chronic back pain'},
 {'first_utterance': '[SR_START] [PAT] : Doctor, I have been feeling Wrist pain since the previous monday. What is wrong with me? [SR_END]',
  'label': 'air embolism'},
 {'first_utterance': '[SR_

In [12]:
len(self_report)

1367

In [13]:
self_report

[{'first_utterance': "[SR_START] [PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SR_END]",
  'label': 'central retinal artery or vein occlusion'},
 {'first_utterance': '[SR_START] [PAT] : Doctor, I have Shoulder pain. I feel terrible. Would you please tell me what could be its cause? [SR_END]',
  'label': 'degenerative disc disease'},
 {'first_utterance': '[SR_START] [PAT] : Hi! Doctor, I am suffering from Foreign body sensation in eye. Would you please tell me what could be its cause? [SR_END]',
  'label': 'diabetic retinopathy'},
 {'first_utterance': "[SR_START] [PAT] : Doctor, I am dealing with Low back pain. I can't sleep well because of this, and in the daytime feels lethargic. Please help me.  [SR_END]",
  'label': 'chronic back pain'},
 {'first_utterance': '[SR_START] [PAT] : Doctor, I have been feeling Wrist pain since the previous monday. What is wrong with me? [SR_END]',
  'label': 'air embolism'},
 {'first_utterance': '[SR_

In [14]:
total_dialog_sheet[0]

{'dialog': "[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK].",
 'label': 'central retinal artery or vein occlusion'}

In [15]:
total_sep_dialog = []

for i in range(len(total_dialog_sheet)):
  temp_dlog = total_dialog_sheet[i]['dialog']
  total_sep_dialog.append(temp_dlog)

In [16]:
total_sep_dialog[0]

"[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK]."

In [17]:
len(total_sep_dialog)

1367

In [18]:
self_report[0]

{'first_utterance': "[SR_START] [PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SR_END]",
 'label': 'central retinal artery or vein occlusion'}

In [19]:
self_report_utterance = []

for i in range(len(self_report)):
  temp_utterance = self_report[i]['first_utterance']
  self_report_utterance.append(temp_utterance)


In [20]:
self_report_utterance[0]

"[SR_START] [PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SR_END]"

In [21]:
len(self_report_utterance)

1367

In [22]:
self_report_utterance[237]

"[SR_START] [PAT] : Hello doctor, I have experienced Leg pain. Would you please tell me its cause? I can't sleep all night due to this. Please help me doctor. [SR_END]"

In [23]:
total_sep_dialog[237]

"[PAT] : Hello doctor, I have experienced Leg pain. Would you please tell me its cause? I can't sleep all night due to this. Please help me doctor. [SEP] [DOC] : OK, I wish you didn't have to go through that pain. Leg pain is a common symptom. What matters most is the symptoms associated with it. Have you gone through Seizures? [SEP] [PAT] : True, sometimes I have got Seizures. [SEP] [DOC] : Have you experienced Disturbance of memory? [SEP] [PAT] : Literally, I am suffering from it. [SEP] [DOC] : Do you have Problems with movement? [SEP] [PAT] : Exactly, I have got Problems with movement. [SEP] [DOC] : Alright, you may have [MASK]."

In [24]:
class MyOwnDataset_complete(InMemoryDataset):
    def __init__(self, root, transform = None, pre_transform = None, pre_filter = None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data_complete_1367.pt']

In [25]:
dataset_joint_kg = MyOwnDataset_complete(root = '/content/drive/MyDrive/Colab Notebooks/27/full_dialog/one_disease/joint_graph/')
dataset_joint_kg

MyOwnDataset_complete(1367)

In [26]:
data_joint_kg = []
for i in dataset_joint_kg:
    data_joint_kg.append(i)

In [27]:
data_joint_kg

[Data(x=[8, 768], edge_index=[2, 20], edge_attr=[20], y=[1]),
 Data(x=[12, 768], edge_index=[2, 42], edge_attr=[42], y=[1]),
 Data(x=[3, 768], edge_index=[2, 2], edge_attr=[2], y=[1]),
 Data(x=[7, 768], edge_index=[2, 12], edge_attr=[12], y=[1]),
 Data(x=[12, 768], edge_index=[2, 110], edge_attr=[110], y=[1]),
 Data(x=[5, 768], edge_index=[2, 6], edge_attr=[6], y=[1]),
 Data(x=[11, 768], edge_index=[2, 30], edge_attr=[30], y=[1]),
 Data(x=[7, 768], edge_index=[2, 12], edge_attr=[12], y=[1]),
 Data(x=[8, 768], edge_index=[2, 30], edge_attr=[30], y=[1]),
 Data(x=[7, 768], edge_index=[2, 12], edge_attr=[12], y=[1]),
 Data(x=[10, 768], edge_index=[2, 42], edge_attr=[42], y=[1]),
 Data(x=[13, 768], edge_index=[2, 42], edge_attr=[42], y=[1]),
 Data(x=[12, 768], edge_index=[2, 110], edge_attr=[110], y=[1]),
 Data(x=[5, 768], edge_index=[2, 12], edge_attr=[12], y=[1]),
 Data(x=[6, 768], edge_index=[2, 12], edge_attr=[12], y=[1]),
 Data(x=[11, 768], edge_index=[2, 42], edge_attr=[42], y=[1]),
 

In [28]:
len(data_joint_kg)

1367

In [29]:
with open("/content/drive/MyDrive/Colab Notebooks/27/hierarchical_classification/group_disease_info/dialog_id_group_map2.p", "rb") as f:
  dialog_id_group_map2 = pickle.load(f)

dialog_id_group_map2

{1059: 4,
 19510: 3,
 25630: 1,
 1467: 6,
 5780: 8,
 26258: 5,
 22733: 2,
 1564: 2,
 29211: 4,
 26235: 2,
 16427: 4,
 16512: 8,
 17308: 8,
 16360: 0,
 15081: 3,
 15386: 4,
 27441: 5,
 9641: 3,
 27768: 5,
 21778: 7,
 18706: 2,
 7857: 2,
 29589: 6,
 15191: 4,
 22535: 4,
 6047: 3,
 20116: 7,
 26439: 0,
 11928: 8,
 20634: 3,
 3590: 7,
 3726: 3,
 7899: 4,
 9125: 8,
 27079: 1,
 7995: 5,
 21990: 5,
 12775: 7,
 86: 0,
 19263: 6,
 19699: 2,
 19386: 0,
 23606: 6,
 14328: 7,
 14382: 3,
 27516: 5,
 17279: 3,
 20111: 6,
 19480: 7,
 28758: 8,
 7769: 0,
 5013: 3,
 26904: 7,
 19940: 2,
 3996: 1,
 12006: 2,
 6762: 8,
 2874: 8,
 9180: 0,
 18054: 6,
 12207: 3,
 17117: 0,
 20286: 1,
 13025: 4,
 791: 3,
 10394: 3,
 26863: 7,
 11005: 3,
 9671: 2,
 11134: 5,
 15276: 5,
 16094: 2,
 6624: 0,
 6097: 7,
 16311: 3,
 339: 1,
 16364: 5,
 19563: 0,
 4346: 0,
 27942: 7,
 7828: 1,
 13246: 0,
 16560: 7,
 19534: 2,
 120: 6,
 8868: 1,
 1947: 3,
 28436: 0,
 5509: 6,
 24423: 4,
 14480: 4,
 13768: 5,
 22417: 2,
 21493: 8,
 

In [30]:
with open("/content/drive/MyDrive/Colab Notebooks/27/hierarchical_classification/group_disease_info/disease_relative_position.p", "rb") as f:
  disease_relative_position = pickle.load(f)

disease_relative_position  

{1: 0,
 21: 1,
 24: 2,
 26: 3,
 27: 4,
 28: 5,
 49: 6,
 84: 7,
 85: 8,
 87: 9,
 13: 0,
 19: 1,
 42: 2,
 44: 3,
 51: 4,
 52: 5,
 53: 6,
 54: 7,
 80: 8,
 88: 9,
 6: 0,
 8: 1,
 10: 2,
 16: 3,
 32: 4,
 39: 5,
 56: 6,
 57: 7,
 60: 8,
 74: 9,
 12: 0,
 14: 1,
 20: 2,
 23: 3,
 33: 4,
 48: 5,
 63: 6,
 72: 7,
 73: 8,
 89: 9,
 4: 0,
 17: 1,
 22: 2,
 25: 3,
 35: 4,
 36: 5,
 41: 6,
 43: 7,
 61: 8,
 67: 9,
 0: 0,
 2: 1,
 3: 2,
 38: 3,
 47: 4,
 50: 5,
 55: 6,
 59: 7,
 62: 8,
 71: 9,
 7: 0,
 15: 1,
 29: 2,
 30: 3,
 37: 4,
 46: 5,
 78: 6,
 79: 7,
 83: 8,
 86: 9,
 5: 0,
 31: 1,
 45: 2,
 64: 3,
 65: 4,
 66: 5,
 68: 6,
 70: 7,
 76: 8,
 77: 9,
 9: 0,
 11: 1,
 18: 2,
 34: 3,
 40: 4,
 58: 5,
 69: 6,
 75: 7,
 81: 8,
 82: 9}

In [31]:
with open("/content/drive/MyDrive/Colab Notebooks/27/hierarchical_classification/group_disease_info/disease_to_group_mapping.p", "rb") as f:
  disease_to_group_mapping = pickle.load(f)

disease_to_group_mapping  

{1: 0,
 21: 0,
 24: 0,
 26: 0,
 27: 0,
 28: 0,
 49: 0,
 84: 0,
 85: 0,
 87: 0,
 13: 1,
 19: 1,
 42: 1,
 44: 1,
 51: 1,
 52: 1,
 53: 1,
 54: 1,
 80: 1,
 88: 1,
 6: 2,
 8: 2,
 10: 2,
 16: 2,
 32: 2,
 39: 2,
 56: 2,
 57: 2,
 60: 2,
 74: 2,
 12: 3,
 14: 3,
 20: 3,
 23: 3,
 33: 3,
 48: 3,
 63: 3,
 72: 3,
 73: 3,
 89: 3,
 4: 4,
 17: 4,
 22: 4,
 25: 4,
 35: 4,
 36: 4,
 41: 4,
 43: 4,
 61: 4,
 67: 4,
 0: 5,
 2: 5,
 3: 5,
 38: 5,
 47: 5,
 50: 5,
 55: 5,
 59: 5,
 62: 5,
 71: 5,
 7: 6,
 15: 6,
 29: 6,
 30: 6,
 37: 6,
 46: 6,
 78: 6,
 79: 6,
 83: 6,
 86: 6,
 5: 7,
 31: 7,
 45: 7,
 64: 7,
 65: 7,
 66: 7,
 68: 7,
 70: 7,
 76: 7,
 77: 7,
 9: 8,
 11: 8,
 18: 8,
 34: 8,
 40: 8,
 58: 8,
 69: 8,
 75: 8,
 81: 8,
 82: 8}

In [32]:
with open("/content/drive/MyDrive/Colab Notebooks/27/hierarchical_classification/group_disease_info/group_number_to_disease2.p", "rb") as f:
  group_number_to_disease2 = pickle.load(f)

group_number_to_disease2  

{0: [1, 21, 24, 26, 27, 28, 49, 84, 85, 87],
 1: [13, 19, 42, 44, 51, 52, 53, 54, 80, 88],
 2: [6, 8, 10, 16, 32, 39, 56, 57, 60, 74],
 3: [12, 14, 20, 23, 33, 48, 63, 72, 73, 89],
 4: [4, 17, 22, 25, 35, 36, 41, 43, 61, 67],
 5: [0, 2, 3, 38, 47, 50, 55, 59, 62, 71],
 6: [7, 15, 29, 30, 37, 46, 78, 79, 83, 86],
 7: [5, 31, 45, 64, 65, 66, 68, 70, 76, 77],
 8: [9, 11, 18, 34, 40, 58, 69, 75, 81, 82]}

In [33]:
with open("/content/drive/MyDrive/Colab Notebooks/27/hierarchical_classification/group_disease_info/group_to_number_map.p", "rb") as f:
  group_to_number_map = pickle.load(f)

group_to_number_map  

{1: 0, 4: 1, 5: 2, 6: 3, 7: 4, 12: 5, 13: 6, 14: 7, 19: 8}

In [34]:
tokenizer  = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
bert_model = AutoModel.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
bert_model

Downloading:   0%|          | 0.00/198 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/462 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [35]:
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
p = {
        'additional_special_tokens' : ['[DOC]', '[PAT]', '[SR_START]', '[SR_END]']
    }

tokenizer.add_special_tokens(p)
print(tokenizer.convert_tokens_to_ids('[DOC]'))
print(tokenizer.convert_tokens_to_ids('[PAT]'))
print(tokenizer.convert_tokens_to_ids('[SR_START]'))
print(tokenizer.convert_tokens_to_ids('[SR_END]'))
print(len(tokenizer))
print(bert_model.resize_token_embeddings(len(tokenizer)))
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
[1, 3, 0, 2, 4]
30522
30523
30524
30525
30526
Embedding(30526, 768)
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[DOC]', '[PAT]', '[SR_START]', '[SR_END]']
[1, 3, 0, 2, 4, 30522, 30523, 30524, 30525]


In [36]:
len(data_joint_kg)

1367

In [37]:
dialog_id_group_map2

{1059: 4,
 19510: 3,
 25630: 1,
 1467: 6,
 5780: 8,
 26258: 5,
 22733: 2,
 1564: 2,
 29211: 4,
 26235: 2,
 16427: 4,
 16512: 8,
 17308: 8,
 16360: 0,
 15081: 3,
 15386: 4,
 27441: 5,
 9641: 3,
 27768: 5,
 21778: 7,
 18706: 2,
 7857: 2,
 29589: 6,
 15191: 4,
 22535: 4,
 6047: 3,
 20116: 7,
 26439: 0,
 11928: 8,
 20634: 3,
 3590: 7,
 3726: 3,
 7899: 4,
 9125: 8,
 27079: 1,
 7995: 5,
 21990: 5,
 12775: 7,
 86: 0,
 19263: 6,
 19699: 2,
 19386: 0,
 23606: 6,
 14328: 7,
 14382: 3,
 27516: 5,
 17279: 3,
 20111: 6,
 19480: 7,
 28758: 8,
 7769: 0,
 5013: 3,
 26904: 7,
 19940: 2,
 3996: 1,
 12006: 2,
 6762: 8,
 2874: 8,
 9180: 0,
 18054: 6,
 12207: 3,
 17117: 0,
 20286: 1,
 13025: 4,
 791: 3,
 10394: 3,
 26863: 7,
 11005: 3,
 9671: 2,
 11134: 5,
 15276: 5,
 16094: 2,
 6624: 0,
 6097: 7,
 16311: 3,
 339: 1,
 16364: 5,
 19563: 0,
 4346: 0,
 27942: 7,
 7828: 1,
 13246: 0,
 16560: 7,
 19534: 2,
 120: 6,
 8868: 1,
 1947: 3,
 28436: 0,
 5509: 6,
 24423: 4,
 14480: 4,
 13768: 5,
 22417: 2,
 21493: 8,
 

In [38]:
all_group_list = []
dialog_id_list = []
group_dialog_id_list = {}
for key in dialog_id_group_map2:
  print("key : ", key, " group : ", dialog_id_group_map2[key])
  group_number = dialog_id_group_map2[key]
  dlog_number = key
  all_group_list.append(group_number)
  dialog_id_list.append(dlog_number)

  temp_dlog = []
  temp_dlog.append(dlog_number)

  if(group_number in group_dialog_id_list):
    group_dialog_id_list[group_number].extend(temp_dlog)
  else:
    group_dialog_id_list[group_number] = temp_dlog


key :  1059  group :  4
key :  19510  group :  3
key :  25630  group :  1
key :  1467  group :  6
key :  5780  group :  8
key :  26258  group :  5
key :  22733  group :  2
key :  1564  group :  2
key :  29211  group :  4
key :  26235  group :  2
key :  16427  group :  4
key :  16512  group :  8
key :  17308  group :  8
key :  16360  group :  0
key :  15081  group :  3
key :  15386  group :  4
key :  27441  group :  5
key :  9641  group :  3
key :  27768  group :  5
key :  21778  group :  7
key :  18706  group :  2
key :  7857  group :  2
key :  29589  group :  6
key :  15191  group :  4
key :  22535  group :  4
key :  6047  group :  3
key :  20116  group :  7
key :  26439  group :  0
key :  11928  group :  8
key :  20634  group :  3
key :  3590  group :  7
key :  3726  group :  3
key :  7899  group :  4
key :  9125  group :  8
key :  27079  group :  1
key :  7995  group :  5
key :  21990  group :  5
key :  12775  group :  7
key :  86  group :  0
key :  19263  group :  6
key :  19699  g

In [39]:
net_dlog = 0
for key in group_dialog_id_list:
  print("Key : ", key, " : ", len(group_dialog_id_list[key]))
  net_dlog += len(group_dialog_id_list[key])

print("Total dlogs : ", net_dlog)  

Key :  4  :  140
Key :  3  :  148
Key :  1  :  163
Key :  6  :  151
Key :  8  :  131
Key :  5  :  165
Key :  2  :  155
Key :  0  :  131
Key :  7  :  183
Total dlogs :  1367


In [40]:
all_group_list

[4,
 3,
 1,
 6,
 8,
 5,
 2,
 2,
 4,
 2,
 4,
 8,
 8,
 0,
 3,
 4,
 5,
 3,
 5,
 7,
 2,
 2,
 6,
 4,
 4,
 3,
 7,
 0,
 8,
 3,
 7,
 3,
 4,
 8,
 1,
 5,
 5,
 7,
 0,
 6,
 2,
 0,
 6,
 7,
 3,
 5,
 3,
 6,
 7,
 8,
 0,
 3,
 7,
 2,
 1,
 2,
 8,
 8,
 0,
 6,
 3,
 0,
 1,
 4,
 3,
 3,
 7,
 3,
 2,
 5,
 5,
 2,
 0,
 7,
 3,
 1,
 5,
 0,
 0,
 7,
 1,
 0,
 7,
 2,
 6,
 1,
 3,
 0,
 6,
 4,
 4,
 5,
 2,
 8,
 4,
 1,
 7,
 6,
 3,
 6,
 8,
 6,
 8,
 4,
 6,
 5,
 7,
 6,
 3,
 7,
 7,
 0,
 5,
 1,
 5,
 5,
 5,
 3,
 8,
 6,
 1,
 6,
 1,
 8,
 3,
 4,
 7,
 2,
 8,
 7,
 0,
 1,
 1,
 4,
 3,
 4,
 5,
 6,
 5,
 0,
 3,
 3,
 7,
 1,
 8,
 4,
 8,
 6,
 1,
 4,
 5,
 5,
 6,
 7,
 2,
 0,
 7,
 5,
 5,
 0,
 7,
 7,
 2,
 5,
 4,
 0,
 0,
 5,
 3,
 5,
 8,
 7,
 7,
 3,
 8,
 7,
 3,
 8,
 7,
 1,
 0,
 8,
 2,
 4,
 5,
 2,
 6,
 2,
 1,
 4,
 6,
 2,
 7,
 1,
 4,
 6,
 8,
 7,
 5,
 1,
 0,
 8,
 7,
 7,
 0,
 5,
 6,
 2,
 4,
 2,
 8,
 8,
 2,
 3,
 1,
 6,
 6,
 3,
 4,
 4,
 7,
 6,
 2,
 7,
 6,
 7,
 4,
 7,
 0,
 4,
 2,
 2,
 8,
 4,
 1,
 1,
 6,
 3,
 7,
 3,
 5,
 6,
 5,
 5,
 0,
 5,
 1,
 1,
 8,
 0,


In [41]:
data_combined = []
dialog_id_data = {}
for i in range(len(data_joint_kg)):
    temp_list = []
    temp_list.append(data_joint_kg[i])
#     temp_list.append(input_ids[i,:])
#     temp_list.append(attention_mask[i,:])
    temp_list.append(total_sep_dialog[i])
    temp_list.append(self_report_utterance[i])
    temp_list.append(all_group_list[i])

    temp_dlog_id = dialog_id_list[i]
    dialog_id_data[temp_dlog_id] = temp_list
    # temp_list.append(dialog_id_list[i])
    data_combined.append(temp_list)

In [42]:
group_train_dialog_id = {}
group_valid_dialog_id = {}
group_test_dialog_id = {}

for key in group_dialog_id_list:
  print("Key : ",key, " len : ", len(group_dialog_id_list[key]))
  temp_group_len = len(group_dialog_id_list[key])
  temp_train_len = int(0.7*temp_group_len)
  temp_valid_len = int(0.1*temp_group_len)

  temp_train_idx_begin = 0
  temp_train_idx_end = temp_train_idx_begin + temp_train_len

  temp_valid_idx_begin = temp_train_idx_end
  temp_valid_idx_end = temp_valid_idx_begin + temp_valid_len

  temp_test_idx_begin = temp_valid_idx_end
  
  temp_group = key

  temp_dlog_list = group_dialog_id_list[key]

  temp_train_list = temp_dlog_list[temp_train_idx_begin : temp_train_idx_end]
  temp_valid_list = temp_dlog_list[temp_valid_idx_begin : temp_valid_idx_end]
  temp_test_list = temp_dlog_list[temp_test_idx_begin : ]

  print("Group len : ", temp_group_len)
  print("Train begin index : ", temp_train_idx_begin)
  print("Train end index : ", temp_train_idx_end)
  print("Valid begin index : ", temp_valid_idx_begin)
  print("Valid end index : ", temp_valid_idx_end)
  print("Test begin index : ", temp_test_idx_begin)
  print("Train list length : ", len(temp_train_list))
  print("Valid list length : ", len(temp_valid_list))
  print("Test list length : ", len(temp_test_list))

  group_train_dialog_id[temp_group] = temp_train_list
  group_valid_dialog_id[temp_group] = temp_valid_list
  group_test_dialog_id[temp_group] = temp_test_list

  print()




Key :  4  len :  140
Group len :  140
Train begin index :  0
Train end index :  98
Valid begin index :  98
Valid end index :  112
Test begin index :  112
Train list length :  98
Valid list length :  14
Test list length :  28

Key :  3  len :  148
Group len :  148
Train begin index :  0
Train end index :  103
Valid begin index :  103
Valid end index :  117
Test begin index :  117
Train list length :  103
Valid list length :  14
Test list length :  31

Key :  1  len :  163
Group len :  163
Train begin index :  0
Train end index :  114
Valid begin index :  114
Valid end index :  130
Test begin index :  130
Train list length :  114
Valid list length :  16
Test list length :  33

Key :  6  len :  151
Group len :  151
Train begin index :  0
Train end index :  105
Valid begin index :  105
Valid end index :  120
Test begin index :  120
Train list length :  105
Valid list length :  15
Test list length :  31

Key :  8  len :  131
Group len :  131
Train begin index :  0
Train end index :  91
Vali

In [43]:
len(data_combined)

1367

In [44]:
len(data_combined[0])

4

In [45]:
data_combined[0]

[Data(x=[8, 768], edge_index=[2, 20], edge_attr=[20], y=[1]),
 "[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK].",
 "[SR_START] [PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SR_END]",
 4]

In [46]:
data_combined[0][1]

"[PAT] : Doctor, I see Spots or clouds in my vision. I can't see anything clearly. I feel helpless. [SEP] [DOC] : I understand how you feel. I need to ask a few questions to diagnose your disease. Have you got Diminished vision? [SEP] [PAT] : True, I am also dealing with Diminished vision. [SEP] [DOC] : Do you have Symptoms of eye? [SEP] [PAT] : Yes, I have got Symptoms of eye. [SEP] [DOC] : Have you felt Pain in eye? [SEP] [PAT] : Indeed, I am suffering from Pain in eye. [SEP] [DOC] : Your symptoms indicate that you have [MASK]."

In [47]:
# random.shuffle(data_combined)

In [48]:
# data_combined_train = data_combined[:957]
# len(data_combined_train)

In [49]:
# data_combined_valid = data_combined[957:1094]
# len(data_combined_valid)

In [50]:
# data_combined_test = data_combined[1094:]
# len(data_combined_test)

In [51]:
data_combined_train = []

for j in group_train_dialog_id:
  temp_group = j
  temp_dlog_list = group_train_dialog_id[temp_group]
  
  for k in temp_dlog_list:
    temp_info = dialog_id_data[k]
    data_combined_train.append(temp_info)

len(data_combined_train)

953

In [52]:
data_combined_valid = []

for j in group_valid_dialog_id:
  temp_group = j
  temp_dlog_list = group_valid_dialog_id[temp_group]

  for k in temp_dlog_list:
    temp_info = dialog_id_data[k]
    data_combined_valid.append(temp_info)

print(len(data_combined_valid))    

134


In [53]:
data_combined_test = []

for j in group_test_dialog_id:
  temp_group = j
  temp_dlog_list = group_test_dialog_id[temp_group]

  for k in temp_dlog_list:
    temp_info = dialog_id_data[k]
    data_combined_test.append(temp_info)

print(len(data_combined_test))    

280


In [54]:
groupwise_train_data = {}
for i in range(len(data_combined_train)):
  temp_data = data_combined_train[i]
  temp_group = temp_data[3]
  temp_list = []
  temp_list.append(temp_data)
  if(temp_group in groupwise_train_data):
    groupwise_train_data[temp_group].extend(temp_list)
  else:
    groupwise_train_data[temp_group] = temp_list



In [55]:
groupwise_train_data.keys()

dict_keys([4, 3, 1, 6, 8, 5, 2, 0, 7])

In [56]:
for i in range(9):
  print(i, " : ",len(groupwise_train_data[i]))

0  :  91
1  :  114
2  :  108
3  :  103
4  :  98
5  :  115
6  :  105
7  :  128
8  :  91


In [57]:
groupwise_valid_data = {}
for i in range(len(data_combined_valid)):
  temp_data = data_combined_valid[i]
  temp_group = temp_data[3]
  temp_list = []
  temp_list.append(temp_data)

  if(temp_group in groupwise_valid_data):
    groupwise_valid_data[temp_group].extend(temp_list)
  else:
    groupwise_valid_data[temp_group] = temp_list  

In [58]:
groupwise_valid_data.keys()

dict_keys([4, 3, 1, 6, 8, 5, 2, 0, 7])

In [59]:
for i in range(9):
  print(i," : ",len(groupwise_valid_data[i]))

0  :  13
1  :  16
2  :  15
3  :  14
4  :  14
5  :  16
6  :  15
7  :  18
8  :  13


In [60]:
group0_train_data = groupwise_train_data[0]
group0_valid_data = groupwise_valid_data[0]

print(len(group0_train_data))
print(len(group0_valid_data))

91
13


In [61]:
group1_train_data = groupwise_train_data[1]
group1_valid_data = groupwise_valid_data[1]

print(len(group1_train_data))
print(len(group1_valid_data))

114
16


In [62]:
group2_train_data = groupwise_train_data[2]
group2_valid_data = groupwise_valid_data[2]

print(len(group2_train_data))
print(len(group2_valid_data))

108
15


In [63]:
group3_train_data = groupwise_train_data[3]
group3_valid_data = groupwise_valid_data[3]

print(len(group3_train_data))
print(len(group3_valid_data))

103
14


In [64]:
group4_train_data = groupwise_train_data[4]
group4_valid_data = groupwise_valid_data[4]

print(len(group4_train_data))
print(len(group4_valid_data))

98
14


In [65]:
group5_train_data = groupwise_train_data[5]
group5_valid_data = groupwise_valid_data[5]

print(len(group5_train_data))
print(len(group5_valid_data))

115
16


In [66]:
group6_train_data = groupwise_train_data[6]
group6_valid_data = groupwise_valid_data[6]

print(len(group6_train_data))
print(len(group6_valid_data))

105
15


In [67]:
group7_train_data = groupwise_train_data[7]
group7_valid_data = groupwise_valid_data[7]

print(len(group7_train_data))
print(len(group7_valid_data))

128
18


In [68]:
group8_train_data = groupwise_train_data[8]
group8_valid_data = groupwise_valid_data[8]

print(len(group8_train_data))
print(len(group8_valid_data))

91
13


In [69]:
joint_data_after_shuffling_train = []
dialog_after_shuffling_train = []
self_report_after_shuffling_train = []

group_train_data = group2_train_data

for i in range(len(group_train_data)):
  temp_data = group_train_data[i]
  temp_joint_graph = temp_data[0]
  temp_dialog = temp_data[1]
  temp_self_report = temp_data[2]

  joint_data_after_shuffling_train.append(temp_joint_graph)
  dialog_after_shuffling_train.append(temp_dialog)
  self_report_after_shuffling_train.append(temp_self_report)  




In [70]:
print(len(joint_data_after_shuffling_train))
print(len(dialog_after_shuffling_train))
print(len(self_report_after_shuffling_train))

108
108
108


In [71]:
joint_data_after_shuffling_valid = []
dialog_after_shuffling_valid = []
self_report_after_shuffling_valid = []

group_valid_data = group2_valid_data

for i in range(len(group_valid_data)):
  temp_data = group_valid_data[i]
  temp_joint_graph = temp_data[0]
  temp_dialog = temp_data[1]
  temp_self_report = temp_data[2]

  joint_data_after_shuffling_valid.append(temp_joint_graph)
  dialog_after_shuffling_valid.append(temp_dialog)
  self_report_after_shuffling_valid.append(temp_self_report)  


In [72]:
print(len(joint_data_after_shuffling_valid))
print(len(dialog_after_shuffling_valid))
print(len(self_report_after_shuffling_valid))

15
15
15


In [73]:
enc_train = tokenizer(dialog_after_shuffling_train, padding='max_length', max_length=512, truncation=True, return_tensors = 'pt')
enc_train

{'input_ids': tensor([[    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        ...,
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [74]:
enc_valid = tokenizer(dialog_after_shuffling_valid, padding='max_length', max_length=512, truncation=True, return_tensors= 'pt')
enc_valid

{'input_ids': tensor([[    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        ...,
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [75]:
enc_self_report_train = tokenizer(self_report_after_shuffling_train, padding = 'max_length', max_length = 512, truncation=True, return_tensors= 'pt' )
enc_self_report_train

{'input_ids': tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [76]:
enc_self_report_valid = tokenizer(self_report_after_shuffling_valid, padding = 'max_length', max_length = 512, truncation = True, return_tensors = 'pt')
enc_self_report_valid

{'input_ids': tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [77]:
# x1 - Joint Graph data
# x2 - input ids dialog
# x3 - attention mask dialog
# x4 - input ids self report
# x5 - attention mask self report

class Dataset_Knowledge_Linear_SelfReport(Dataset):
    def __init__(self, x1, x2, x3, x4, x5):
        self.x1 = x1
        self.x2 = x2
        self.x3 = x3
        self.x4 = x4
        self.x5 = x5
        
    
    def __len__(self):
        return len(self.x1)
    
    def __getitem__(self, idx):
        x1_idx = self.x1[idx]
        x2_idx = self.x2[idx]
        x3_idx = self.x3[idx]
        x4_idx = self.x4[idx]
        x5_idx = self.x5[idx]
        
        
        return x1_idx, x2_idx, x3_idx, x4_idx, x5_idx
        

In [78]:
train_loader = DataLoader(Dataset_Knowledge_Linear_SelfReport(joint_data_after_shuffling_train, enc_train['input_ids'], enc_train['attention_mask'], enc_self_report_train['input_ids'], enc_self_report_train['attention_mask']), batch_size = 16, shuffle = True)
train_loader




In [79]:
valid_loader = DataLoader(Dataset_Knowledge_Linear_SelfReport(joint_data_after_shuffling_valid, enc_valid['input_ids'], enc_valid['attention_mask'], enc_self_report_valid['input_ids'], enc_self_report_valid['attention_mask']), batch_size = 16, shuffle=False)
valid_loader



In [80]:
for step,data in enumerate(train_loader):
    print("Step : ", step)
    print("Data : ", data)
    print()

Step :  0
Data :  [DataBatch(x=[136, 768], edge_index=[2, 344], edge_attr=[344], y=[16], batch=[136], ptr=[17]), tensor([[    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        ...,
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
  

In [81]:
for step,data in enumerate(valid_loader):
    print("Step : ", step)
    print("Data : ", data)
    print("Label : ", data[0].y)
    print()

Step :  0
Data :  [DataBatch(x=[126, 768], edge_index=[2, 318], edge_attr=[318], y=[15], batch=[126], ptr=[16]), tensor([[    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        ...,
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0],
        [    2, 30523,    30,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([[    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
        ...,
        [    2, 30524, 30523,  ...,     0,     0,     0],
        [    2, 30524, 30523,  ...,     0,     0,     0],
  

In [82]:
class CustomBert_SelfReport(torch.nn.Module):
    def __init__(self):
        super(CustomBert_SelfReport, self).__init__()
        self.bert = bert_model
    
    def forward(self, ids_self_report, mask_self_report):
        output_encoding = self.bert(ids_self_report, mask_self_report)
        
        cls = output_encoding['last_hidden_state'].permute(1,0,2)[0]
        
        return cls

In [83]:
class CustomBert_Dialog(torch.nn.Module):
    def __init__(self):
        super( CustomBert_Dialog, self).__init__()
        self.bert = bert_model
    
    def forward(self, ids_dialog, mask_dialog):
        output_encoding = self.bert(ids_dialog, mask_dialog)
        
        cls = output_encoding['last_hidden_state'].permute(1,0,2)[0]
        
        return cls

In [84]:
class GAT_joint(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, args):
        super(GAT_joint, self).__init__()
        
        self.conv1 = GATConv(input_dim, hidden_dim, heads = args['heads'])
        self.conv2 = GATConv(args['heads']*hidden_dim, hidden_dim, heads = args['heads'])
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        x = self.conv1(x, edge_index)
        x = F.dropout(F.relu(x), p=args['dropout'], training = self.training)
        
        x = self.conv2(x, edge_index)
        x = F.dropout(F.relu(x), p=args['dropout'], training = self.training)
        
        x = global_mean_pool(x, batch)
        x = F.dropout(F.relu(x), p=args['dropout'], training = self.training)
        
        return x
        

In [85]:
768 + 384 

1152

In [86]:
768 + 768 + 384

1920

In [87]:
class Attention(torch.nn.Module):
  def __init__(self, hidden_dim1, hidden_dim2, proj_dim):
    super(Attention, self).__init__()

    self.W = torch.nn.Linear(hidden_dim1, proj_dim, bias=False)
    self.U = torch.nn.Linear(hidden_dim2, proj_dim, bias=False)
    self.V = torch.nn.Linear(proj_dim,1, bias=False)

  def forward(self, k, q):
      # k shape = [batch, 2, 768]
      # q shape = [batch, 384]
      # print("Q shape : ", q.shape)
      x = torch.unsqueeze(q, dim = 1)
      # print("Q shape : ", x.shape)
      out = F.tanh(self.W(k) + self.U(x))
      # print("Out shape : ", out.shape)
      out2 = self.V(out)
      # print("Out2 shape : ", out2.shape)
      out2 = torch.squeeze(out2, dim=-1)
      # print("Out2 shape : ", out2.shape)
      attn = F.softmax(out2, dim=-1)
      # print("Attention shape : ", attn.shape)
      attn = torch.unsqueeze(attn, dim=1)
      # print("Attention shape : ", attn.shape)
      context = torch.bmm(attn, k)
      # print("Context shape : ", context.shape)
      context = torch.squeeze(context, dim=1)
      # print("Context shape : ", context.shape)

      return context




In [88]:
z5 = torch.tensor([9,1,2,3,4,21])
y5 = torch.argmax(z5)
y5

tensor(5)

In [89]:
y5 = y5.tolist()
y5

5

In [90]:
type(y5)

int

In [91]:
class Linear_Knowledge_SelfReport(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, args, hidden_dim1, hidden_dim2, proj_dim):
        super(Linear_Knowledge_SelfReport, self).__init__()
        
        self.custom_bert_self_report = CustomBert_SelfReport()
        self.custom_bert_dialog = CustomBert_Dialog()
        self.gat_joint = GAT_joint(input_dim, hidden_dim, args)
        # self.linear1 = torch.nn.Linear(1920, 384)
        # self.linear2 = torch.nn.Linear(384, output_dim)
        # self.temp_linear = torch.nn.Linear(1920, output_dim)

        self.attention = Attention(hidden_dim1, hidden_dim2, proj_dim)
        self.temp_linear = torch.nn.Linear(1152, output_dim)
        
        

    def forward(self, data, input_ids_dialog, attention_mask_dialog, input_ids_self_report, attention_mask_self_report):
        bert_output_self_report = self.custom_bert_self_report(input_ids_self_report, attention_mask_self_report)
        bert_output_dialog = self.custom_bert_dialog(input_ids_dialog, attention_mask_dialog)
        gat_output = self.gat_joint(data)

        # print("Bert output self report : ", bert_output_self_report.shape)
        bert_output_self_report = torch.unsqueeze(bert_output_self_report, dim=1)
        # print("Bert output self report : ", bert_output_self_report.shape)

        # print("Bert output dialog : ", bert_output_dialog.shape)
        bert_output_dialog = torch.unsqueeze(bert_output_dialog, dim = 1)
        # print("Bert output dialog : ", bert_output_dialog.shape)

        bert_concat = torch.concat((bert_output_self_report, bert_output_dialog), dim=1)

        # print("Bert concat shape : ", bert_concat.shape)

        context = self.attention(bert_concat, gat_output)

        # print("Context shape : ", context.shape)
        
        final = torch.cat((context, gat_output), dim=1)


        
        # final = self.linear1(final)
        # final = F.relu(final)
        
        # final = self.linear2(final)

        final = self.temp_linear(final)
        
        return final

In [92]:
args = {
    'heads' : 3,
    'dropout' : 0.5
}

In [93]:
model = Linear_Knowledge_SelfReport(input_dim = 768, hidden_dim = 128, output_dim = 10, args = args, hidden_dim1 = 768, hidden_dim2 = 384, proj_dim = 64)
model

Linear_Knowledge_SelfReport(
  (custom_bert_self_report): CustomBert_SelfReport(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30526, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768,

In [94]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [95]:
model = model.to(device)
model

Linear_Knowledge_SelfReport(
  (custom_bert_self_report): CustomBert_SelfReport(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30526, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768,

In [96]:
model.parameters()

<generator object Module.parameters at 0x7fe1918b7250>

In [97]:
total_parameters = sum(p.numel() for p in model.parameters())
total_parameters/1e6

110.015306

In [98]:
total_train_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_train_parameters/1e6

110.015306

In [99]:
for i,name in enumerate(model.named_parameters()):
    print(i," : ",name[0])

0  :  custom_bert_self_report.bert.embeddings.word_embeddings.weight
1  :  custom_bert_self_report.bert.embeddings.position_embeddings.weight
2  :  custom_bert_self_report.bert.embeddings.token_type_embeddings.weight
3  :  custom_bert_self_report.bert.embeddings.LayerNorm.weight
4  :  custom_bert_self_report.bert.embeddings.LayerNorm.bias
5  :  custom_bert_self_report.bert.encoder.layer.0.attention.self.query.weight
6  :  custom_bert_self_report.bert.encoder.layer.0.attention.self.query.bias
7  :  custom_bert_self_report.bert.encoder.layer.0.attention.self.key.weight
8  :  custom_bert_self_report.bert.encoder.layer.0.attention.self.key.bias
9  :  custom_bert_self_report.bert.encoder.layer.0.attention.self.value.weight
10  :  custom_bert_self_report.bert.encoder.layer.0.attention.self.value.bias
11  :  custom_bert_self_report.bert.encoder.layer.0.attention.output.dense.weight
12  :  custom_bert_self_report.bert.encoder.layer.0.attention.output.dense.bias
13  :  custom_bert_self_report.b

In [100]:
for p in model.parameters():
    p.requires_grad = False

In [101]:
total_train_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_train_parameters/1e6

0.0

In [102]:
cnt=0
for name, param in model.named_parameters():
    if(param.requires_grad == False and cnt>=199):
        param.requires_grad = True
        print(name)
#         print()
    cnt+=1    

gat_joint.conv1.att_src
gat_joint.conv1.att_dst
gat_joint.conv1.bias
gat_joint.conv1.lin_src.weight
gat_joint.conv2.att_src
gat_joint.conv2.att_dst
gat_joint.conv2.bias
gat_joint.conv2.lin_src.weight
attention.W.weight
attention.U.weight
attention.V.weight
temp_linear.weight
temp_linear.bias


In [103]:
total_train_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_train_parameters/1e6

0.529994

In [104]:
1e-3

0.001

In [105]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = torch.nn.CrossEntropyLoss()


In [106]:
def train(loader):
    model.train()
    
    for data in loader:
        optimizer.zero_grad()
        
        graph_data = data[0]
        input_ids_dialog = data[1]
        attention_mask_dialog = data[2]
        input_ids_self_report = data[3]
        attention_mask_self_report = data[4]
        
        
        graph_data = graph_data.to(device)
        input_ids_dialog = input_ids_dialog.to(device)
        attention_mask_dialog = attention_mask_dialog.to(device)
        input_ids_self_report = input_ids_self_report.to(device)
        attention_mask_self_report = attention_mask_self_report.to(device)
        
        
        out = model(graph_data, input_ids_dialog, attention_mask_dialog, input_ids_self_report, attention_mask_self_report)
        
        z = graph_data.y
        ground_truth = z.long()
        
        ground_truth_list = ground_truth.tolist()
        ground_truth_convert = []

        for j in range(len(ground_truth_list)):
          temp_id = disease_relative_position[ground_truth_list[j]]
          ground_truth_convert.append(temp_id)

        ground_truth_convert = torch.tensor(ground_truth_convert)
        ground_truth_convert = ground_truth_convert.to(device)

          

        loss = criterion(out, ground_truth_convert)
        
        loss.backward()
        optimizer.step()

In [107]:
def test(loader):
    model.eval()

    # y_true = []
    # y_pred = []
    
    correct = 0
    for data in loader:
        with torch.no_grad():
            
            graph_data = data[0]
            input_ids_dialog = data[1]
            attention_mask_dialog = data[2]
            input_ids_self_report = data[3]
            attention_mask_self_report = data[4]
            

            graph_data = graph_data.to(device)
            input_ids_dialog = input_ids_dialog.to(device)
            attention_mask_dialog = attention_mask_dialog.to(device)
            input_ids_self_report = input_ids_self_report.to(device)
            attention_mask_self_report = attention_mask_self_report.to(device)
            

            out = model(graph_data, input_ids_dialog, attention_mask_dialog, input_ids_self_report, attention_mask_self_report)

            z = graph_data.y
            ground_truth = z.long()
            
            ground_truth_list = ground_truth.tolist()
            ground_truth_convert = []

            for j in range(len(ground_truth_list)):
              temp_id = disease_relative_position[ground_truth_list[j]]
              ground_truth_convert.append(temp_id)

            ground_truth_convert = torch.tensor(ground_truth_convert)
            ground_truth_convert = ground_truth_convert.to(device) 
            
            disease_pred = out.argmax(dim=1)
            
            correct += int((disease_pred == ground_truth_convert).sum())

            # true_label = graph_data.y

            # y_true.extend(true_label.tolist())
            # y_pred.extend(pred.tolist())

            
    # return correct/len(loader.dataset), y_true, y_pred
    return correct/len(loader.dataset)        

In [108]:
q1=torch.tensor([1,2,3,4,5])
q1

tensor([1, 2, 3, 4, 5])

In [109]:
q2=torch.tensor([2,2,2,4,6])
q2

tensor([2, 2, 2, 4, 6])

In [110]:
int((q1 == q2).sum())

2

In [111]:
768+384

1152

In [112]:
for epoch in range(25):
    train(train_loader)
    train_acc = test(train_loader)
    valid_acc = test(valid_loader)
    print(f'Epoch : {epoch : 03d}, Train acc : {train_acc : .4f}, Valid acc : {valid_acc : .4f}')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1949: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch :  00, Train acc :  0.3426, Valid acc :  0.2667
Epoch :  01, Train acc :  0.7315, Valid acc :  0.4667
Epoch :  02, Train acc :  0.7870, Valid acc :  0.7333
Epoch :  03, Train acc :  0.8611, Valid acc :  0.7333
Epoch :  04, Train acc :  0.8611, Valid acc :  0.7333
Epoch :  05, Train acc :  0.8981, Valid acc :  0.7333
Epoch :  06, Train acc :  0.9074, Valid acc :  0.7333
Epoch :  07, Train acc :  0.9444, Valid acc :  0.7333
Epoch :  08, Train acc :  0.9352, Valid acc :  0.7333
Epoch :  09, Train acc :  0.9630, Valid acc :  0.8000
Epoch :  10, Train acc :  0.9537, Valid acc :  0.8000
Epoch :  11, Train acc :  0.9907, Valid acc :  0.8000
Epoch :  12, Train acc :  0.9722, Valid acc :  0.8000
Epoch :  13, Train acc :  0.9630, Valid acc :  0.8000
Epoch :  14, Train acc :  0.9722, Valid acc :  0.7333
Epoch :  15, Train acc :  0.9722, Valid acc :  0.8000
Epoch :  16, Train acc :  1.0000, Valid acc :  0.8667
Epoch :  17, Train acc :  0.9907, Valid acc :  0.7333
Epoch :  18, Train acc :  0.

In [113]:
# for epoch in range(25):
#     train(train_loader)
#     train_acc, y_true_train, y_pred_train = test(train_loader)
#     valid_acc, y_true_valid, y_pred_valid = test(valid_loader)
#     print(f'Epoch : {epoch : 03d}, Train acc : {train_acc : .4f}, Valid acc : {valid_acc : .4f}')

In [114]:
input_dim = 768
hidden_dim = 128
output_dim = 10
hidden_dim1 = 768
hidden_dim2 = 384
proj_dim = 64

In [115]:
dict_save = {}
dict_save['epoch'] = epoch
dict_save['model_state_dict'] = model.state_dict()
dict_save['optimizer_state_dict'] = optimizer.state_dict()
dict_save['args'] = args
dict_save['input_dim'] = input_dim
dict_save['hidden_dim'] = hidden_dim
dict_save['output_dim'] = output_dim
dict_save['hidden_dim1'] = hidden_dim1
dict_save['hidden_dim2'] = hidden_dim2
dict_save['proj_dim'] = proj_dim

path = "/content/drive/MyDrive/Colab Notebooks/27/hierarchical_classification/one_disease/one_disease_saved_model/dict_save_group2_disease.pt"

torch.save(dict_save, path)


In [116]:
# test_acc, y_true_test, y_pred_test = test(test_loader)
# print("Test acc : ", test_acc)

In [117]:
# print("=========Using Sklearn======")
# print("\nConfusion matrix\n", confusion_matrix(y_true_test, y_pred_test))
# print("\nAccuracy\n", accuracy_score(y_true_test, y_pred_test))
# print("\nF1 score\n", f1_score(y_true_test, y_pred_test, average = 'macro'))
# print("\nJaccard similarity\n", jaccard_score(y_true_test, y_pred_test, average = 'macro'))

In [118]:
# confusion_matrix = confusion_matrix(y_true_test, y_pred_test)
# confusion_matrix

In [119]:
# with open("/content/drive/MyDrive/Colab Notebooks/27/self_report_full_dialog_joint_graph/having_first_utterance/ground_truth_test.p", "wb") as f:
#   pickle.dump(y_true_test, f)

In [120]:
# with open("/content/drive/MyDrive/Colab Notebooks/27/self_report_full_dialog_joint_graph/having_first_utterance/predicted_test.p", "wb") as f:
#   pickle.dump(y_pred_test, f)

In [121]:
# with open("/content/drive/MyDrive/Colab Notebooks/27/self_report_full_dialog_joint_graph/having_first_utterance/confusion_matrix.p", "wb") as f:
#   pickle.dump(confusion_matrix, f)